In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
train=pd.read_csv("/content/drive/MyDrive/Minor Final/Dataset/Final_Dataset/Potato_Red.csv")
train.head()

,Commodity,Date,Average
0,Potato Red,6/16/2013,20.5
1,Potato Red,6/17/2013,20.5
2,Potato Red,6/18/2013,19.5
3,Potato Red,6/19/2013,20.5
4,Potato Red,6/20/2013,20.5


In [3]:
train['Date']  = train['Date'].apply(pd.to_datetime)

In [4]:
train.head()

,Commodity,Date,Average
0,Potato Red,2013-06-16,20.5
1,Potato Red,2013-06-17,20.5
2,Potato Red,2013-06-18,19.5
3,Potato Red,2013-06-19,20.5
4,Potato Red,2013-06-20,20.5


In [5]:
(train["Date"] - train.Date.shift(1))

0         NaT
1      1 days
2      1 days
3      1 days
4      1 days
        ...  
3597   1 days
3598   1 days
3599   1 days
3600   1 days
3601   1 days
Name: Date, Length: 3602, dtype: timedelta64[ns]

In [6]:
(train["Date"] - train.Date.shift(1)).value_counts()

1 days    3486
2 days      94
3 days      12
4 days       4
5 days       3
8 days       1
7 days       1
Name: Date, dtype: int64

In [7]:
train.isna().sum()

Commodity    0
Date         0
Average      0
dtype: int64

In [8]:
px.line(
    data_frame=train,
    x="Date",
    y="Average"
)

In [9]:
resampled_df = train.resample("7D", on ='Date' , ).sum()
px.line(
    data_frame=resampled_df,
#     x="Date",
    y="Average"
)

<ipython-input-9-9834976e19c2>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [10]:
resampled_df

,Average
Date,
2013-06-16,123.0
2013-06-23,93.0
2013-06-30,139.0
2013-07-07,94.0
2013-07-14,141.5
...,...
2023-08-27,390.3
2023-09-03,393.7
2023-09-10,400.8


In [11]:
from sklearn.model_selection import train_test_split

y_train=train['Average']
# Splitting 'y_train' into training and testing sets without shuffling
y_train, y_test = train_test_split(y_train, test_size=0.2, shuffle=False)

In [12]:
# Training size:
print("Train set has {} observations.".format(len(y_train)))
# Test size:
print("Test set has {} observations.".format(len(y_test)))

Train set has 2881 observations.
Test set has 721 observations.


In [13]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
y_train=scaler.fit_transform(y_train.values.reshape(-1,1))
y_test=scaler.transform(y_test.values.reshape(-1,1))

In [14]:
# Sequence Data Preparation
SEQUENCE_SIZE = 10

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(0 , len(obs) - seq_size):
        window = obs[i:i + seq_size]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Using y_test only
x_train, y_train = to_sequences(SEQUENCE_SIZE, y_train)
x_test, y_test = to_sequences(SEQUENCE_SIZE, y_test)

# Setup data loaders for batch
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

<ipython-input-14-08401c7e56ea>:12: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)



In [15]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [16]:
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: using Google CoLab
Using device: cpu


In [17]:
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()
        self.encoder = nn.Linear(input_dim, d_model )
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers,enable_nested_tensor=True)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])
        return x

model = TransformerModel(
    input_dim = 1,
    d_model = 32,
    nhead = 16,
    num_layers = 2,
    dropout= 0.05,
).to(device)

In [18]:
model

TransformerModel(
  (encoder): Linear(in_features=1, out_features=32, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.05, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=32, out_features=1, bias=True)
)

In [19]:
from torch.optim.lr_scheduler import ConstantLR
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)
scheduler = ConstantLR(optimizer)
epochs = 100
early_stop_count = 0
min_val_loss = float('inf')
train_losses = []
validation_losses = []
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step()
    validation_losses.append(val_loss)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    # if early_stop_count >= 5:
    #     print("Early stopping!")
    #     break
    print(f"Epoch {epoch + 1}/{epochs},Train Loss {loss.item()} Validation Loss: {val_loss:.4f}")

Epoch 1/100,Train Loss 0.03265923634171486 Validation Loss: 0.0538
Epoch 2/100,Train Loss 0.0004470762505661696 Validation Loss: 0.0341
Epoch 3/100,Train Loss 0.005159144755452871 Validation Loss: 0.0314
Epoch 4/100,Train Loss 0.008585525676608086 Validation Loss: 0.0244
Epoch 5/100,Train Loss 0.00033106986666098237 Validation Loss: 0.0198
Epoch 6/100,Train Loss 0.06442377716302872 Validation Loss: 0.0668
Epoch 7/100,Train Loss 0.006626307964324951 Validation Loss: 0.0207
Epoch 8/100,Train Loss 0.0007876405725255609 Validation Loss: 0.0255
Epoch 9/100,Train Loss 0.029083361849188805 Validation Loss: 0.0291
Epoch 10/100,Train Loss 0.10452282428741455 Validation Loss: 0.0699
Epoch 11/100,Train Loss 0.02006862871348858 Validation Loss: 0.0265
Epoch 12/100,Train Loss 0.0017798002809286118 Validation Loss: 0.0231
Epoch 13/100,Train Loss 0.08414042741060257 Validation Loss: 0.0404
Epoch 14/100,Train Loss 0.0030683199875056744 Validation Loss: 0.0200
Epoch 15/100,Train Loss 0.0006656316109001

In [ ]:
# px.line(
#     x = range(len(train_losses)),
#     y = train_losses,
# )

In [20]:
torch.save(model,"/content/drive/MyDrive/Minor Final/Transformer_Final model/Potato_Red.pth")

In [ ]:
model = torch.load("/content/drive/MyDrive/Minor Final/Transformer_Final model/Potato_Red.pth", map_location=torch.device('cpu'))

In [ ]:
criterion = nn.MSELoss()
model.eval()
predictions = []
valid_loss = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.tolist())
        validation_loss = criterion(outputs, y_batch)
        valid_loss.append(validation_loss.item())

# Convert predictions to a 2D numpy array
predictions = np.array(predictions).reshape(-1, 1)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)

# Calculate R^2 score
r2 = r2_score(y_test, predictions)

# Print scores
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

RMSE: 0.1559
MAE: 0.1036
MSE: 0.0243
R^2 Score: 0.9794
